# Using Spark SQL with Python on CSV

In [23]:
# Quick view of the structure of the incoming csv file
import pandas as pd
pandf = pd.read_csv("Uber-Jan-Feb-FOIL.csv", header=0)
pandf.head()

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537


In [ ]:
pandf

In [51]:
# Call read function in the sqlcontext to read the incoming file 
# as `CSV` format using the provided package as the options..

df = sqlContext.read.format("com.databricks.spark.csv") \
    .options(header='true', inferschema='true').load("Uber-Jan-Feb-FOIL.csv")

In [52]:
df.take(5)

[Row(dispatching_base_number=u'B02512', date=u'1/1/2015', active_vehicles=190, trips=1132),
 Row(dispatching_base_number=u'B02765', date=u'1/1/2015', active_vehicles=225, trips=1765),
 Row(dispatching_base_number=u'B02764', date=u'1/1/2015', active_vehicles=3427, trips=29421),
 Row(dispatching_base_number=u'B02682', date=u'1/1/2015', active_vehicles=945, trips=7679),
 Row(dispatching_base_number=u'B02617', date=u'1/1/2015', active_vehicles=1228, trips=9537)]

## Register a `TempTable` to make the `Spark SQL` much more like SQL.
e.g. we can use the registered temp name behave like SQL Table Name.

In [19]:
# Register the TempTable name and display the structure / schema.
df.registerTempTable("uber")
df

DataFrame[dispatching_base_number: string, date: string, active_vehicles: int, trips: int]

## `printSchema()`

In [26]:
df.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: integer (nullable = true)
 |-- trips: integer (nullable = true)



## SELECT `DISTINCT` `dispatching_base_number`

In [22]:
sqlContext.sql("SELECT DISTINCT dispatching_base_number from uber").collect()

[Row(dispatching_base_number=u'B02598'),
 Row(dispatching_base_number=u'B02764'),
 Row(dispatching_base_number=u'B02765'),
 Row(dispatching_base_number=u'B02617'),
 Row(dispatching_base_number=u'B02682'),
 Row(dispatching_base_number=u'B02512')]

In [24]:
# Done in Pandas
pandf.dispatching_base_number.unique()

array(['B02512', 'B02765', 'B02764', 'B02682', 'B02617', 'B02598'], dtype=object)

# Support for advanced SQL functions

Spark SQL has support for advanced SQL features and functions. Some are highlighed below:

### What `Bases` are the busiest?
> Notice the quote mark on column_names are not approstrophes ( ' ), but accent mark ( ` ) (top left of keyboard!)

In [49]:
sqlContext.sql("""SELECT DISTINCT(`dispatching_base_number`),
                   SUM(`trips`) AS TripCount FROM uber
                   GROUP BY `dispatching_base_number`
                   ORDER BY TripCount DESC
                """).show()

+-----------------------+---------+
|dispatching_base_number|TripCount|
+-----------------------+---------+
|                 B02764|  1914449|
|                 B02617|   725025|
|                 B02682|   662509|
|                 B02598|   540791|
|                 B02765|   193670|
|                 B02512|    93786|
+-----------------------+---------+



### What `Dates` are the busiest?
> Notice the quote mark can also sometimes be ommited!

In [48]:
sqlContext.sql("""SELECT DISTINCT (date),
                   SUM(trips) AS TripCount FROM uber
                   GROUP BY date
                   ORDER BY TripCount DESC LIMIT 10
                """).show()

+---------+---------+
|     date|TripCount|
+---------+---------+
|2/20/2015|   100915|
|2/14/2015|   100345|
|2/21/2015|    98380|
|2/13/2015|    98024|
|1/31/2015|    92257|
|2/15/2015|    89401|
|2/27/2015|    88806|
|2/19/2015|    88757|
|2/28/2015|    88181|
| 2/6/2015|    85940|
+---------+---------+

